**Programa para el envío masivo de recibos de mantenimiento por correo**

In [37]:
!pip install PyPDF2 pandas
import os
import re
import pandas as pd
from PyPDF2 import PdfReader, PdfWriter
from google.colab import drive
from datetime import datetime
import time
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication
from email.utils import formatdate
import traceback

# Montar Google Drive
drive.mount('/content/drive')
base_path = '/content/drive/MyDrive/Condominio El Mirador I/RECIBOS 2025/Programa de envío masivo de correos'

# Crear carpetas base si no existen
os.makedirs(f'{base_path}/Recibos_Origen', exist_ok=True)
os.makedirs(f'{base_path}/Recibos_Individuales', exist_ok=True)
os.makedirs(f'{base_path}/Base_Datos', exist_ok=True)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [38]:
# BLOQUE 1: CONFIGURACIÓN INICIAL
# ============================================
print("🟢 BLOQUE 1: Configuración inicial")
mes = input("Ingrese el mes (ej: Julio): ").strip().capitalize()
año = input("Ingrese el año (ej: 2025): ").strip()
periodo = f"{mes}_{año}"

# Crear carpeta para el periodo
periodo_folder = f'{base_path}/Recibos_Individuales/{periodo}/'
os.makedirs(periodo_folder, exist_ok=True)
print(f"✅ Periodo configurado: {mes} {año}")
print(f"📁 Carpeta creada: {periodo_folder}")

# Ruta del archivo Excel
db_path = f'{base_path}/Base_Datos/base_datos.xlsx'

🟢 BLOQUE 1: Configuración inicial
Ingrese el mes (ej: Julio): Septiembre
Ingrese el año (ej: 2025): 2025
✅ Periodo configurado: Septiembre 2025
📁 Carpeta creada: /content/drive/MyDrive/Condominio El Mirador I/RECIBOS 2025/Programa de envío masivo de correos/Recibos_Individuales/Septiembre_2025/


In [39]:
# ============================================
# BLOQUE 2: EXTRAER RECIBOS INDIVIDUALES
# ============================================
print("\n🟢 BLOQUE 2: Extraer recibos individuales")

# Buscar archivo PDF en Recibos_Origen
pdf_files = [f for f in os.listdir(f'{base_path}/Recibos_Origen') if f.lower().endswith('.pdf')]
if not pdf_files:
    print("⚠️ Error: No se encontraron archivos PDF en Recibos_Origen")
    print("Por favor sube el archivo PDF y vuelve a ejecutar este bloque")
else:
    input_pdf = f'{base_path}/Recibos_Origen/{pdf_files[0]}'
    print(f"📄 Procesando archivo: {pdf_files[0]}")

    with open(input_pdf, 'rb') as file:
        reader = PdfReader(file)
        total_pages = len(reader.pages)
        print(f"🔢 Total de páginas: {total_pages}")

        departamentos_procesados = []

        for page_num in range(total_pages):
            page = reader.pages[page_num]
            text = page.extract_text()

            # Buscar patrón de departamento (ej: 701B)
            dpto_match = re.search(r'(\d+[AB])\s*$', text, re.MULTILINE)

            if dpto_match:
                dpto = dpto_match.group(1)
                writer = PdfWriter()
                writer.add_page(page)

                # Guardar como PDF individual
                output_pdf = f'{periodo_folder}{dpto}.pdf'
                with open(output_pdf, 'wb') as output_file:
                    writer.write(output_file)

                departamentos_procesados.append(dpto)
                print(f"✅ {dpto}", end=' ', flush=True)

        print(f"\n\n📁 Total departamentos procesados: {len(departamentos_procesados)}")
        print(f"📍 Recibos guardados en: {periodo_folder}")


🟢 BLOQUE 2: Extraer recibos individuales
📄 Procesando archivo: SETIEMBRE 2025.xlsx - RECIBOS.pdf
🔢 Total de páginas: 126
✅ 101A ✅ 503A ✅ 903A ✅ 1303A ✅ 303B ✅ 703B ✅ 1103B ✅ 102A ✅ 504A ✅ 904A ✅ 1304A ✅ 304B ✅ 704B ✅ 1104B ✅ 201A ✅ 601A ✅ 1001A ✅ 1401A ✅ 401B ✅ 801B ✅ 1201B ✅ 202A ✅ 602A ✅ 1002A ✅ 1402A ✅ 402B ✅ 802B ✅ 1202B ✅ 203A ✅ 603A ✅ 1003A ✅ 1403A ✅ 403B ✅ 803B ✅ 1203B ✅ 204A ✅ 604A ✅ 1004A ✅ 1404A ✅ 404B ✅ 804B ✅ 1204B ✅ 301A ✅ 701A ✅ 1101A ✅ 101B ✅ 501B ✅ 901B ✅ 1301B ✅ 302A ✅ 702A ✅ 1102A ✅ 102B ✅ 502B ✅ 902B ✅ 1302B ✅ 303A ✅ 703A ✅ 1103A ✅ 103B ✅ 503B ✅ 903B ✅ 1303B ✅ 304A ✅ 704A ✅ 1104A ✅ 104B ✅ 504B ✅ 904B ✅ 1304B ✅ 401A ✅ 801A ✅ 1201A ✅ 201B ✅ 601B ✅ 1001B ✅ 1401B ✅ 402A ✅ 802A ✅ 1202A ✅ 202B ✅ 602B ✅ 1002B ✅ 1402B ✅ 403A ✅ 803A ✅ 1203A ✅ 203B ✅ 603B ✅ 1003B ✅ 1403B ✅ 404A ✅ 804A ✅ 1204A ✅ 204B ✅ 604B ✅ 1004B ✅ 1404B ✅ 501A ✅ 901A 

✅ 1301A ✅ 301B ✅ 701B ✅ 1101B ✅ 502A ✅ 902A ✅ 1302A ✅ 302B ✅ 702B ✅ 1102B 

📁 Total departamentos procesados: 110
📍 Recibos guardados en: /content/drive/MyDrive/Condominio El Mirador I/RECIBOS 2025/Programa de envío masivo de correos/Recibos_Individuales/Septiembre_2025/


In [40]:
# ============================================
# BLOQUE 3: VERIFICAR BASE DE DATOS
# ============================================
print("\n🟢 BLOQUE 3: Verificar base de datos")

# Cargar base de datos
if os.path.exists(db_path):
    try:
        # Leer Excel con encabezado en la primera fila
        df = pd.read_excel(db_path)
        print("🔍 Base de datos existente cargada")

        # Verificar estructura
        print("Columnas detectadas:", df.columns.tolist())

        # Renombrar columnas a nombres estandarizados
        rename_map = {}
        column_mapping = {
            'DPTO': 'Departamento',
            'NOMBRES Y APELLIDOS': 'Propietario',
            'CORREO_1': 'Email',
            'CORREO_2': 'Email_CC',
            'NOMBRE_ARCHIVO': 'Archivo_PDF'
        }

        for original, nuevo in column_mapping.items():
            if original in df.columns:
                rename_map[original] = nuevo

        if rename_map:
            df = df.rename(columns=rename_map)
            print(f"↪️ Columnas renombradas: {rename_map}")

        # Verificar si existen todas las columnas necesarias
        required_columns = ['Departamento', 'Propietario', 'Email', 'Archivo_PDF']
        missing_cols = [col for col in required_columns if col not in df.columns]

        if missing_cols:
            print(f"⚠️ Advertencia: Faltan columnas: {', '.join(missing_cols)}")

            # Añadir columnas faltantes
            for col in missing_cols:
                if col not in df.columns:
                    df[col] = ''
                    print(f"➕ Columna '{col}' añadida")
    except Exception as e:
        print(f"❌ Error al cargar la base de datos: {str(e)}")
        exit()
else:
    print("❌ Error: No se encontró la base de datos")
    exit()

# Mostrar vista previa detallada
print("\n📊 Vista previa de la base de datos (primeras 5 filas):")
print(df.head().to_string(index=False))

print("\n🔍 Verificación de datos:")
print(f"- Total registros: {len(df)}")
print(f"- Emails no vacíos: {df['Email'].notna().sum()}")
print(f"- Archivos PDF asignados: {df['Archivo_PDF'].notna().sum()}")


🟢 BLOQUE 3: Verificar base de datos
🔍 Base de datos existente cargada
Columnas detectadas: ['DPTO', 'NOMBRES Y APELLIDOS', 'DOCUMENTO DE IDENTIDAD', 'CORREO_1', 'CELULAR_1', 'CORREO_2', 'CELULAR_2', 'COCHERA', 'MASCOTAS', 'OBSERVACION', 'NOMBRE_ARCHIVO']
↪️ Columnas renombradas: {'DPTO': 'Departamento', 'NOMBRES Y APELLIDOS': 'Propietario', 'CORREO_1': 'Email', 'CORREO_2': 'Email_CC', 'NOMBRE_ARCHIVO': 'Archivo_PDF'}

📊 Vista previa de la base de datos (primeras 5 filas):
Departamento                      Propietario DOCUMENTO DE IDENTIDAD                    Email   CELULAR_1 Email_CC   CELULAR_2 COCHERA MASCOTAS OBSERVACION Archivo_PDF
        101A MIRTHA RAFAELA VILCABANA OCHANTE               43368397                      NaN   924309163      NaN         NaN     NaN      NaN         NaN    101A.pdf
        102A     MIRTA JESSICA MONROY CARNERO               74648205                      NaN 987 751 256      NaN 982313748.0     NaN      NaN         NaN    102A.pdf
        201A      

In [41]:
# ============================================
# BLOQUE 4: ENVIAR CORREOS
# ============================================
print("\n🟢 BLOQUE 4: Enviar correos")

# Función para ordenar departamentos en el orden específico
def ordenar_departamentos(df):
    # Definir el orden exacto de los departamentos
    orden_especifico = [
        '101A', '102A', '201A', '202A', '203A', '204A', '301A', '302A', '303A', '304A',
        '401A', '402A', '403A', '404A', '501A', '502A', '503A', '504A', '601A', '602A',
        '603A', '604A', '701A', '702A', '703A', '704A', '801A', '802A', '803A', '804A',
        '901A', '902A', '903A', '904A', '1001A', '1002A', '1003A', '1004A', '1101A', '1102A',
        '1103A', '1104A', '1201A', '1202A', '1203A', '1204A', '1301A', '1302A', '1303A', '1304A',
        '1401A', '1402A', '1403A', '1404A', '101B', '102B', '103B', '104B', '201B', '202B',
        '203B', '204B', '301B', '302B', '303B', '304B', '401B', '402B', '403B', '404B',
        '501B', '502B', '503B', '504B', '601B', '602B', '603B', '604B', '701B', '702B',
        '703B', '704B', '801B', '802B', '803B', '804B', '901B', '902B', '903B', '904B',
        '1001B', '1002B', '1003B', '1004B', '1101B', '1102B', '1103B', '1104B', '1201B', '1202B',
        '1203B', '1204B', '1301B', '1302B', '1303B', '1304B', '1401B', '1402B', '1403B', '1404B'
    ]

    # Crear un diccionario de mapeo de departamento a posición
    orden_map = {depto: idx for idx, depto in enumerate(orden_especifico)}

    # Filtrar solo los departamentos que existen en el DataFrame
    df['Departamento'] = df['Departamento'].str.strip().str.upper()
    departamentos_existentes = df['Departamento'].unique()

    # Ordenar el DataFrame según el orden específico
    df['orden'] = df['Departamento'].map(orden_map)
    df = df.sort_values('orden').drop('orden', axis=1).reset_index(drop=True)

    return df

# Función para seleccionar el departamento
def seleccionar_departamento(df_validos):
    # Ordenar el DataFrame según el orden específico
    df_work = ordenar_departamentos(df_validos)

    # Mostrar el primer y último departamento disponible como referencia
    primer_depto = df_work.iloc[0]['Departamento']
    ultimo_depto = df_work.iloc[-1]['Departamento']

    print(f"📋 Rango de departamentos: {primer_depto} a {ultimo_depto}")

    while True:
        # Pedir al usuario que ingrese el departamento de inicio
        departamento = input(
            "Ingrese el departamento desde el cual comenzar el envío\n"
            "(ejemplo: 101A para empezar desde el inicio, o 601A para continuar): "
        ).strip().upper()

        # Si está vacío, usar el primer departamento
        if not departamento:
            print(f"✅ Comenzando desde el primer departamento: {primer_depto}")
            return df_work

        # Buscar el departamento exacto
        if departamento in df_work['Departamento'].values:
            print(f"✅ Departamento encontrado: {departamento}")
            # Encontrar el índice del departamento
            index_departamento = df_work[df_work['Departamento'] == departamento].index[0]
            return df_work.iloc[index_departamento:]

        # Si no existe, buscar el siguiente disponible automáticamente
        print(f"⚠️ El departamento '{departamento}' no se encuentra.")
        print("🔍 Buscando el siguiente departamento disponible...")

        # Buscar en el orden específico
        orden_especifico = [
            '101A', '102A', '201A', '202A', '203A', '204A', '301A', '302A', '303A', '304A',
            '401A', '402A', '403A', '404A', '501A', '502A', '503A', '504A', '601A', '602A',
            '603A', '604A', '701A', '702A', '703A', '704A', '801A', '802A', '803A', '804A',
            '901A', '902A', '903A', '904A', '1001A', '1002A', '1003A', '1004A', '1101A', '1102A',
            '1103A', '1104A', '1201A', '1202A', '1203A', '1204A', '1301A', '1302A', '1303A', '1304A',
            '1401A', '1402A', '1403A', '1404A', '101B', '102B', '103B', '104B', '201B', '202B',
            '203B', '204B', '301B', '302B', '303B', '304B', '401B', '402B', '403B', '404B',
            '501B', '502B', '503B', '504B', '601B', '602B', '603B', '604B', '701B', '702B',
            '703B', '704B', '801B', '802B', '803B', '804B', '901B', '902B', '903B', '904B',
            '1001B', '1002B', '1003B', '1004B', '1101B', '1102B', '1103B', '1104B', '1201B', '1202B',
            '1203B', '1204B', '1301B', '1302B', '1303B', '1304B', '1401B', '1402B', '1403B', '1404B'
        ]

        # Encontrar la posición del departamento en el orden específico
        try:
            pos_ingresado = orden_especifico.index(departamento)
        except ValueError:
            # Si no está en la lista, encontrar el primer departamento mayor
            pos_ingresado = -1

        # Buscar el siguiente departamento disponible
        siguiente_depto = None
        if pos_ingresado >= 0:
            # Buscar desde la posición siguiente
            for depto in orden_especifico[pos_ingresado + 1:]:
                if depto in df_work['Departamento'].values:
                    siguiente_depto = depto
                    break
        else:
            # Buscar el primer departamento mayor al ingresado
            for depto in orden_especifico:
                if depto > departamento and depto in df_work['Departamento'].values:
                    siguiente_depto = depto
                    break

        if siguiente_depto:
            print(f"✅ Continuando desde: {siguiente_depto}")
            index_departamento = df_work[df_work['Departamento'] == siguiente_depto].index[0]
            return df_work.iloc[index_departamento:]
        else:
            # Si no hay departamentos mayores, mostrar mensaje
            print(f"❌ No hay departamentos después de '{departamento}'")
            print("💡 Por favor ingrese un departamento válido del rango mostrado.")

# Limpiar y validar emails
print("\n🧹 Realizando limpieza de datos...")
df['Email'] = df['Email'].fillna('').astype(str).str.strip()
df['Propietario'] = df['Propietario'].fillna('').astype(str).str.strip()
df['Email_CC'] = df.get('Email_CC', '').fillna('').astype(str).str.strip()

# Filtrar registros con email válido
email_mask = (df['Email'] != '') & (df['Email'].str.contains('@', na=False))
df_validos = df[email_mask].copy()
emails_invalidos = df[~email_mask]

# Imprimir resumen de los datos
print("\n📊 RESUMEN DE DATOS:")
print(f"📋 Total registros: {len(df)}")
print(f"✉️  Correos válidos: {len(df_validos)}")
print(f"⚠️  Correos inválidos: {len(emails_invalidos)}")

# Mostrar primeros 5 correos válidos para verificación
if not df_validos.empty:
    print("\n✅ Primeros 5 correos válidos:")
    print(df_validos[['Departamento', 'Propietario', 'Email', 'Email_CC']].head().to_string(index=False))

if df_validos.empty:
    print("\n❌ ERROR: No hay registros con email válido")
    exit()

# Seleccionar departamento de inicio
df_departamento = seleccionar_departamento(df_validos)

if df_departamento is not None and not df_departamento.empty:
    primer_depto_envio = df_departamento.iloc[0]['Departamento']
    print(f"\n🚀 INICIANDO ENVÍO DESDE: {primer_depto_envio}")
    print(f"📧 Total de correos a enviar: {len(df_departamento)}")

    # Mostrar los primeros 3 departamentos que se enviarán
    print("\n📋 Primeros 3 departamentos en la cola de envío:")
    for i in range(min(3, len(df_departamento))):
        depto = df_departamento.iloc[i]['Departamento']
        propietario = df_departamento.iloc[i]['Propietario']
        print(f"   {i+1}. {depto} - {propietario}")
else:
    print("\n❌ No se pudo determinar desde qué departamento comenzar")
    exit()

# =============================================================================
# CONFIGURACIÓN Y ENVÍO DE CORREOS
# =============================================================================

# Configuración SMTP
print("\n🔐 CONFIGURACIÓN SMTP")
remitente = 'condominiomiradorvenezuela@gmail.com'
password = 'ljje pgay rakn ldkk'  # Contraseña de aplicación
asunto = f'Recibo de Mantenimiento - {mes} {año}'

# Límites y temporizador
limite_destinatarios = 500  # Límite diario de Gmail
pausa_entre_envios = 15  # Segundos entre envíos
destinatarios_enviados = 0  # Contador de destinatarios

# Estadísticas de envío
exitosos = 0
fallidos = 0
errores_archivos = 0
errores_envio = 0

try:
    print("\n🔌 CONECTANDO CON SERVIDOR SMTP...")
    server = smtplib.SMTP('smtp.gmail.com', 587, timeout=30)
    server.starttls()
    server.login(remitente, password)
    print("🔑 AUTENTICACIÓN EXITOSA")

    total = len(df_departamento)
    print(f"\n✉️ INICIANDO ENVÍO DE {total} CORREOS...")
    print(f"⏱️ Pausa entre envíos: {pausa_entre_envios} segundos")

    for index, row in df_departamento.iterrows():
        dpto = row['Departamento']
        propietario = row['Propietario'] or 'propietario'
        email = row['Email']
        email_cc = row.get('Email_CC', '')
        archivo_pdf = row['Archivo_PDF']
        pdf_path = f"{periodo_folder}{archivo_pdf}"

        print(f"\n➡️ [{index+1}/{total}] Enviando a {propietario} <{email}>")
        if email_cc:
            print(f"   📩 CC: {email_cc}")

        try:
            # Crear mensaje
            msg = MIMEMultipart()
            msg['From'] = remitente
            msg['To'] = email

            # Añadir CC si existe
            if email_cc:
                msg['Cc'] = email_cc

            msg['Subject'] = asunto
            msg['Date'] = formatdate(localtime=True)

            # Cuerpo personalizado
            cuerpo = f"""
            Estimado(a) {propietario},

            Adjunto encontrará el recibo de mantenimiento correspondiente a su departamento {dpto}.

            Periodo: {mes} {año}
            Fecha de vencimiento: 15 de {mes} (tener en cuenta que, posterior a la fecha, se carga una mora de 15 soles)

            Saludos cordiales,
            Administración del Condominio
            """
            msg.attach(MIMEText(cuerpo, 'plain'))

            # Verificar y adjuntar PDF
            if os.path.exists(pdf_path):
                with open(pdf_path, 'rb') as f:
                    part = MIMEApplication(f.read(), Name=archivo_pdf)

                part.add_header('Content-Disposition', 'attachment',
                               filename=f"Recibo_{dpto}_{mes}_{año}.pdf")
                msg.attach(part)
                print(f"📎 Adjunto: {archivo_pdf}")
            else:
                errores_archivos += 1
                print(f"❌ ARCHIVO NO ENCONTRADO: {pdf_path}")
                print("⚠️ Enviando correo sin adjunto...")

            # Preparar lista de destinatarios (incluyendo CC)
            destinatarios = [email]
            if email_cc:
                # Manejar múltiples correos en CC (separados por coma o punto y coma)
                cc_list = [e.strip() for e in re.split(r'[;,]\s*', email_cc) if e.strip()]
                destinatarios.extend(cc_list)

            # Contar destinatarios
            total_destinatarios = len(destinatarios)
            destinatarios_enviados += total_destinatarios

            # Verificar límite diario
            if destinatarios_enviados > limite_destinatarios:
                print(f"⛔ LÍMITE DIARIO EXCEDIDO: {destinatarios_enviados}/{limite_destinatarios} destinatarios")
                print("💡 Soluciones:")
                print("- Espera 24 horas para que se reinicie tu cuota")
                print("- Considera usar un servicio profesional como SendGrid")
                fallidos += 1
                errores_envio += 1
                break

            # Intentar envío con reconexión
            for intento in range(1, 4):  # 3 intentos
                try:
                    server.sendmail(remitente, destinatarios, msg.as_string())
                    exitosos += 1
                    print(f"✅ ENVIADO (Intento {intento}/3)")
                    print(f"📬 Destinatarios enviados: {destinatarios_enviados}/{limite_destinatarios}")
                    break
                except smtplib.SMTPDataError as e:
                    if "Daily user sending limit exceeded" in str(e):
                        print("⛔ LÍMITE DIARIO EXCEDIDO")
                        print("💡 Por favor usa una cuenta de G Suite o intenta mañana")
                        fallidos += 1
                        errores_envio += 1
                        # Salir completamente del proceso
                        raise
                    else:
                        if intento < 3:
                            print(f"⚠️ Error en envío. Reintentando... ({str(e)})")
                            time.sleep(5)  # Esperar antes de reintentar
                            continue
                        else:
                            raise
                except smtplib.SMTPServerDisconnected:
                    if intento < 3:
                        print(f"⚠️ Desconectado. Reconectando... (Intento {intento+1}/3)")
                        server.connect('smtp.gmail.com', 587)
                        server.starttls()
                        server.login(remitente, password)
                        continue
                    else:
                        raise
                except Exception as e:
                    if intento < 3:
                        print(f"⚠️ Error en envío. Reintentando... ({str(e)})")
                        time.sleep(5)  # Esperar antes de reintentar
                        continue
                    else:
                        raise

            # Pausa para evitar sobrecarga
            print(f"⏳ Esperando {pausa_entre_envios} segundos antes del próximo envío...")
            time.sleep(pausa_entre_envios)

        except smtplib.SMTPDataError as e:
            if "Daily user sending limit exceeded" in str(e):
                print("⛔ ERROR CRÍTICO: Límite diario excedido")
                print("🛑 Deteniendo el proceso de envío")
                fallidos += 1
                errores_envio += 1
                break
            else:
                fallidos += 1
                errores_envio += 1
                print(f"❌ ERROR EN ENVÍO: {str(e)}")
                print(f"🛠️ Detalles técnicos:\n{traceback.format_exc()}")
        except Exception as e:
            fallidos += 1
            errores_envio += 1
            print(f"❌ ERROR EN ENVÍO: {str(e)}")
            print(f"🛠️ Detalles técnicos:\n{traceback.format_exc()}")

except Exception as e:
    print(f"❌ ERROR CRÍTICO EN EL SISTEMA: {str(e)}")
    print(f"🛠️ Detalles técnicos:\n{traceback.format_exc()}")

finally:
    # Cerrar conexión SMTP de forma segura
    try:
        server.quit()
        print("🔒 CONEXIÓN SMTP CERRADA")
    except:
        pass

# Reporte final
print("\n📊 REPORTE FINAL DE ENVÍOS")
print(f"✅ CORREOS EXITOSOS: {exitosos}/{len(df_departamento)}")
print(f"❌ CORREOS FALLIDOS: {fallidos}/{len(df_departamento)}")
print(f"📎 ERRORES DE ARCHIVO: {errores_archivos}")
print(f"✉️  ERRORES DE ENVÍO: {errores_envio}")
print(f"👤 DESTINATARIOS ENVIADOS: {destinatarios_enviados}/{limite_destinatarios}")

if fallidos == 0 and errores_archivos == 0:
    print("\n🎉 ¡TODOS LOS CORREOS SE ENVIARON EXITOSAMENTE!")
elif fallidos > 0 or errores_archivos > 0:
    print("\n⚠️ ALGUNOS CORREOS NO PUDIERON ENVIARSE COMPLETAMENTE")
    print("ℹ️ Recomendaciones:")
    print("- Verifica los mensajes de error anteriores")
    print("- Para correos fallidos, intenta reenviar manualmente")
    print("- Verifica la existencia de los archivos PDF en: " + periodo_folder)
    print("- Revisa la configuración de seguridad de tu cuenta Gmail")

    if "Daily user sending limit exceeded" in traceback.format_exc():
        print("\n🔔 IMPORTANTE: Has excedido el límite diario de envíos de Gmail")
        print("💡 Soluciones:")
        print("1. Espera 24 horas para que se reinicie tu cuota (se reinicia a medianoche PST)")
        print("2. Considera usar un servicio profesional como SendGrid para envíos masivos")
        print("3. Divide los envíos en varios días")
        print("4. Verifica tu actividad de Gmail: https://myaccount.google.com/apppasswords")


🟢 BLOQUE 4: Enviar correos

🧹 Realizando limpieza de datos...

📊 RESUMEN DE DATOS:
📋 Total registros: 110
✉️  Correos válidos: 102
⚠️  Correos inválidos: 8

✅ Primeros 5 correos válidos:
Departamento                    Propietario                    Email               Email_CC
        201A      JESSICA ALTAMIRANO AGÜERO    jennysentir@gmail.com                       
        202A    LUISA CRISTINA NARVAEZ ANZA kelly.canturin@gmail.com                       
        204A       YVAN LUIS SAYBAY COTRINA       ylsaybay@gmail.com                       
        301A   ROSANA MARIA ROMERO PINILLOS      tanarp1@hotmail.com                       
        302A SILVIA DEL ROSARIO AVALOS TASA  silvia.1706@hotmail.com Vivilorena93@gmail.com
📋 Rango de departamentos: 201A a 1404B
Ingrese el departamento desde el cual comenzar el envío
(ejemplo: 101A para empezar desde el inicio, o 601A para continuar): 601A
✅ Departamento encontrado: 601A

🚀 INICIANDO ENVÍO DESDE: 601A
📧 Total de correos a enviar:

KeyboardInterrupt: 